In [1]:
import pandas as pd
import csv

In [2]:
data = pd.read_excel('WholePremise_LoadShapeDatav11.xlsx', skiprows=1)
data.head

<bound method NDFrame.head of               City,State Springfield, IL Springfield, IL.1 Springfield, IL.2  \
0                 Sector      Commercial        Commercial        Commercial   
1           Heating Type        Electric          Electric          Electric   
2              Bldg Type   Office, Large     Office, Large     Office, Large   
3                 Region         Midwest           Midwest           Midwest   
4                    Day             HR1               HR2               HR3   
..                   ...             ...               ...               ...   
740  2001-12-28 00:00:00         3.40277           3.56706           3.65192   
741  2001-12-29 00:00:00         3.39528           3.46493           3.52094   
742  2001-12-30 00:00:00         3.86993           4.18424           4.31692   
743  2001-12-31 00:00:00         3.51558            3.5604           3.89187   
744                  NaN               1                 2                 3   

    Sprin

In [3]:
# data = data.filter(like='SAN ANTONIO, TX')

In [4]:
data.head

<bound method NDFrame.head of               City,State Springfield, IL Springfield, IL.1 Springfield, IL.2  \
0                 Sector      Commercial        Commercial        Commercial   
1           Heating Type        Electric          Electric          Electric   
2              Bldg Type   Office, Large     Office, Large     Office, Large   
3                 Region         Midwest           Midwest           Midwest   
4                    Day             HR1               HR2               HR3   
..                   ...             ...               ...               ...   
740  2001-12-28 00:00:00         3.40277           3.56706           3.65192   
741  2001-12-29 00:00:00         3.39528           3.46493           3.52094   
742  2001-12-30 00:00:00         3.86993           4.18424           4.31692   
743  2001-12-31 00:00:00         3.51558            3.5604           3.89187   
744                  NaN               1                 2                 3   

    Sprin

In [5]:
print(data.columns)

Index(['City,State', 'Springfield, IL', 'Springfield, IL.1',
       'Springfield, IL.2', 'Springfield, IL.3', 'Springfield, IL.4',
       'Springfield, IL.5', 'Springfield, IL.6', 'Springfield, IL.7',
       'Springfield, IL.8',
       ...
       'TUCSON, AZ.422', 'TUCSON, AZ.423', 'TUCSON, AZ.424', 'TUCSON, AZ.425',
       'TUCSON, AZ.426', 'TUCSON, AZ.427', 'TUCSON, AZ.428', 'TUCSON, AZ.429',
       'TUCSON, AZ.430', 'TUCSON, AZ.431'],
      dtype='object', length=16369)


In [6]:
#See your choices for locations
def peek_locations(data):
    locations = set(data.columns.values)
    print(locations)
    return locations
    

In [7]:
#Create new DF with like locatios
def filter_location(data, location):
    data = data.filter(like=location)
    return data
    

In [8]:
#peek at your buildings
def peek_buildings(data):
    print(set(data.iloc[2].values))
    return list(set(data.iloc[2].values))


In [9]:
#Input a list of buildings you want to build loadshapes for
def make_buildings_df(data, buildings='All'):
    if buildings =='All':
        buildings = list(set(data.iloc[2].values))
    print(buildings)
    df_dict = {}
    for i in buildings:
        buildings = data.columns[data.iloc[2] == i]
        df = data[buildings]
        #only support electric for now, assume military bases are rural enough that they cannot support nat gas.
        source = df.columns[df.iloc[1]=='Electric']
        df = df[source].dropna()
        df_dict[i] = df
#     print(df_dict)
    return df_dict


In [13]:
#Data Transformer, this function take in a df, sums all hourly load of buildings in df for certain location
def sum_load(df):
    load_shape_arr = [0]*8760
    for building in df.values():
        idx = 0
        rows = len(building)
        cols = len(building.columns)

        for row in range(5,(rows)):
            for col in range((cols)):
    #             print(building.iloc[row,col])
                load_shape_arr[idx] = [building.iloc[row,col]]
                idx+=1
    return load_shape_arr
            


In [14]:
#Driver
place = 'SAN ANTONIO, TX'
locations = peek_locations(data)
data = filter_location(data, place)
buildings = peek_buildings(data)
df = make_buildings_df(data)

{'SAN ANTONIO, TX.214', 'SAN ANTONIO, TX.175', 'SAN ANTONIO, TX.254', 'SAN ANTONIO, TX.204', 'SAN ANTONIO, TX.417', 'SAN ANTONIO, TX.251', 'SAN ANTONIO, TX.377', 'SAN ANTONIO, TX.60', 'SAN ANTONIO, TX.210', 'SAN ANTONIO, TX.131', 'SAN ANTONIO, TX.311', 'SAN ANTONIO, TX.301', 'SAN ANTONIO, TX.244', 'SAN ANTONIO, TX.172', 'SAN ANTONIO, TX.125', 'SAN ANTONIO, TX.294', 'SAN ANTONIO, TX.241', 'SAN ANTONIO, TX.201', 'SAN ANTONIO, TX.300', 'SAN ANTONIO, TX.121', 'SAN ANTONIO, TX.154', 'SAN ANTONIO, TX.320', 'SAN ANTONIO, TX.310', 'SAN ANTONIO, TX.387', 'SAN ANTONIO, TX.73', 'SAN ANTONIO, TX.337', 'SAN ANTONIO, TX.285', 'SAN ANTONIO, TX.213', 'SAN ANTONIO, TX.129', 'SAN ANTONIO, TX.171', 'SAN ANTONIO, TX.335', 'SAN ANTONIO, TX.232', 'SAN ANTONIO, TX.376', 'SAN ANTONIO, TX.404', 'SAN ANTONIO, TX.331', 'SAN ANTONIO, TX.72', 'SAN ANTONIO, TX.352', 'SAN ANTONIO, TX.19', 'SAN ANTONIO, TX.12', 'SAN ANTONIO, TX.326', 'SAN ANTONIO, TX.124', 'SAN ANTONIO, TX.288', 'SAN ANTONIO, TX.258', 'SAN ANTONIO, T

In [21]:
load_shape = sum_load(df)

In [23]:
#Save the loadshape as csv vector
with open('load_shape_'+str(place)+'_.csv', 'w', newline='') as outfile:
    wr = csv.writer(outfile)
    wr.writerows(load_shape)

In [22]:
print(load_shape)

[[1.73054763542685], [1.71052372377363], [1.75600846579595], [1.87005753791693], [1.97723416392719], [2.06354100977992], [2.14260993404345], [2.06063039321165], [1.84701185993652], [1.66345085892474], [1.54242007482833], [1.44601178817359], [1.36986666773787], [1.34622756307849], [1.33629378886197], [1.29820521554325], [1.30136073835999], [1.44391124611946], [1.62044720558878], [1.70291279112908], [1.7412198961772], [1.77070472459797], [1.76211605085378], [1.76311866704717], [1.79429469942221], [1.80296508322126], [1.81345495860956], [1.87868716141544], [1.96692596384605], [2.13155085507508], [2.41045023044173], [2.72809599237779], [3.02078540070512], [3.08630111365662], [2.97472743669579], [2.88762267484769], [2.80964992400619], [2.75112994824466], [2.72085156207762], [2.69040655152674], [2.56868122180748], [2.33518574757033], [2.11023288166861], [1.89990068371563], [1.72901369997172], [1.68622107649934], [1.71486306200125], [1.76934298218227], [1.82802887177982], [1.84338777601196], 

In [37]:
print(type(load_shape))

<class 'list'>


In [26]:
template = pd.read_csv('load_profile_template.csv')
print(template)

      Hour  Load (kW)
0        1        NaN
1        2        NaN
2        3        NaN
3        4        NaN
4        5        NaN
...    ...        ...
8755  8756        NaN
8756  8757        NaN
8757  8758        NaN
8758  8759        NaN
8759  8760        NaN

[8760 rows x 2 columns]
